In [4]:
import time
import tracemalloc
from typing import Dict, List, Tuple
from collections import defaultdict

class Node:
    def __init__(self, item=None, prob=0.0, count=0, utility=0, nu = 0):
        self.item = item
        self.prob = prob
        self.count = count
        self.utility = utility
        self.children = {}
        self.parent = None
        self.nu = nu

class UHUFP_tree:
    def __init__(self):
        self.root = Node()
        self.header_table = defaultdict(list)

    def insert_reorganized_transaction(self, transaction, utilities, sorted_items):
        # print(f"Inserting transaction: {transaction}")
        # print(f"Item utilities: {utilities}")
        # print(f"Sorted items: {sorted_items}")
        
        current_node = self.root
        rtu = sum(utilities.values())
        # print(f"Total utility of transaction: {total_utility}")

        for i, item in enumerate(sorted_items):
            item_prob = transaction[item]
            item_utility = utilities[item]
            
            # print(f"Processing item: {item}")
            # print(f"Item probability: {item_prob}, Item utility: {item_utility}")
            
            if (item, item_prob, item_utility) in current_node.children:
                child_node = current_node.children[(item, item_prob, item_utility)]
                # print(f"Found existing child node for item {item}")
                child_node.count += 1
                items_after = sorted_items[i+1:]
                remaining_utility = sum(utilities.get(it, 0) for it in items_after)
                child_node.nu += rtu - remaining_utility
                # print(f"Updated child node: count={child_node.count}, nu={child_node.nu}")
            else:
                child_node = Node(item=item, prob=item_prob, count=1, utility=item_utility)
                items_after = sorted_items[i+1:]
                remaining_utility = sum(utilities.get(it, 0) for it in items_after)
                child_node.nu = rtu - remaining_utility
                # print(f"Creating new child node: item={item}")
                # print(f"Calculated node utility (nu): {child_node.nu}")
                
                child_node.parent = current_node
                current_node.children[(item, item_prob, item_utility)] = child_node
                self.header_table[item].append(child_node)
            
            current_node = child_node
            # print(f"Current node set to: {current_node.item}")

        # After processing all items, print the final state of the tree
        # print("Current state of the UP-Tree:")
        # self.print_tree()


    
    def insert_reorganized_path(self, transaction, utilities, sorted_items, miu):
        current_node = self.root
        for i, item in enumerate(sorted_items):
            item_prob = transaction[item]
            item_utility = utilities[item]

            # Debug: Print current state
            # print(f"Inserting item: {item}, item_prob: {item_prob}, item_utility: {item_utility}")

            if (item, item_prob, item_utility) in current_node.children:
                child_node = current_node.children[(item, item_prob, item_utility)]
                child_node.count += 1

                # Calculate the utility of items after the current node
                utility_after = sum(miu.get(x, 0)*child_node.count for x in list(sorted_items.keys())[i + 1:])
                # print(f"Utility of items after {item}: {utility_after}")

                # Update node utility
                child_node.nu += sorted_items[item] - utility_after
                # print(f"Updated node {item} nu: {child_node.nu}")
            else:
                # Create a new node
                child_node = Node(item=item, prob=item_prob, count=1, utility=item_utility)

                # Calculate the utility of items after the current node
                utility_after = sum(miu.get(x, 0)*child_node.count for x in list(sorted_items.keys())[i + 1:])
                # print(f"Utility of items after {item}: {utility_after}")

                # Set initial node utility
                child_node.nu = sorted_items[item] - utility_after
                # print(f"Initial node {item} nu: {child_node.nu}")

                child_node.parent = current_node
                current_node.children[(item, item_prob, item_utility)] = child_node
                self.header_table[item].append(child_node)

            current_node = child_node




    def print_header_table(self):
        print("\nHeader Table:")
        for item, nodes in self.header_table.items():
            # print(f"Item: {item}")
            for node in nodes:
                # print(f"  -> Node: Item={node.item}, Support={node.prob}")
                print(f"  -> Node: Item={node.item}, Support={node.prob} , Count={node.count}, Utility={node.utility}")
        print("End of Header Table\n")


    def print_tree(self):
        def print_node(node, level=0):
            indent = ' ' * (level * 4)
            print(f"{indent}Item: {node.item}, Support: {node.prob}, Count: {node.count}, Utility: {node.utility}, Nu: {node.nu}")
            for child in node.children.values():
                print_node(child, level + 1)

        print("UHUFP Tree:")
        print_node(self.root)
        print("End of UHUFP Tree")

    def prune_patterns(self, k, item_twu, minUtil):
        # Compute expected support for each item
        item_exp_sups = {}
        for item, nodes in self.header_table.items():
            exp_sup = sum(node.prob*node.count for node in nodes)
            item_exp_sups[item] = exp_sup

        sorted_items_by_exp_sup = sorted(self.header_table.keys(), key=lambda item: item_exp_sups[item], reverse=True)
        self.header_table = {item: self.header_table[item] for item in sorted_items_by_exp_sup}

        
        # Extract top-k items based on expected support
        filtered_header_table = defaultdict(list)
        for item, nodes in self.header_table.items():
            # filtered_header_table[item] = self.header_table[item]
            k -= 1
            twu = item_twu[item]
            if twu >= minUtil:
                filtered_header_table[item] = self.header_table[item]
            if k <= 0:
                break
        self.header_table = filtered_header_table
        # self.print_header_table()
        # sorted_items_by_twu = sorted(self.header_table.keys(), key=lambda item: item_twu[item], reverse=True)
        # print(sorted_items_by_twu)
        # Rebuild header table sorted by TWU
        # self.header_table = {item: self.header_table[item] for item in sorted_items_by_twu}

    """Remove nodes from the tree that are not present in the updated header table."""
    def prune_tree(self, node):
        items_to_keep = set(self.header_table.keys())
        children_to_keep = {k: v for k, v in node.children.items() if v.item in items_to_keep}
            
        # Update the current node's children
        node.children = children_to_keep
            
        # Recursively prune children
        for child in node.children.values():
            self.prune_tree(child)

class TUHUFP_Growth:


    def __init__(self):
        self.start_timestamp = 0
        self.end_timestamp = 0
        self.database_size = 0
        self.peak_memory_usage = 0
        self.top_k_UHUFP = []
        self.minUtil = 0


    def calculate_expected_support(self, paths, prefix):
        total_expected_support = 0.0
        # paths = self.collect_paths_prefix(prefix)

        for path, count in paths:
            # Tạo tập hợp các item trong path
            path_items = {node[0] for node in path}

            # Kiểm tra xem tất cả các item trong prefix có nằm trong path không
            if all(item in path_items for item in prefix):
                path_support = 1.0
                # Tính expected support cho path bằng cách nhân các xác suất (prob) của các items trong prefix
                for item in prefix:
                    # Tìm item trong path
                    i = next((i for i in path if i[0] == item), None)
                    if i:
                        path_support = round((path_support*i[1]),2)  # Nhân xác suất (prob) của item trong path
                    else:
                        path_support = 0
                        break

                total_expected_support += path_support * count  # Nhân với count của path

        return round(total_expected_support, 2)




    def calculate_utility(self, paths, prefix):
        prefix_utility = 0
        # paths = self.collect_paths_prefix(prefix)

        for path, count in paths:
            # Tạo tập hợp các item trong path
            path_items = {node[0] for node in path}

            # Kiểm tra xem tất cả các item trong prefix có nằm trong path không
            if all(item in path_items for item in prefix):
                sum_utility = 0
                # Tính expected support cho path bằng cách nhân các xác suất (prob) của các items trong prefix
                for item in prefix:
                    # Tìm item trong path
                    i = next((i for i in path if i[0] == item), None)
                    if i:
                        sum_utility += i[2]  # Nhân xác suất (prob) của item trong path
                    else:
                        sum_utility = 0
                        break

                prefix_utility += sum_utility * count  # Nhân với count của path

        return prefix_utility

        
 
    def calculate_twu(self, paths, prefix):
        prefix_twu = 0
        # paths = self.collect_paths_prefix(prefix)

        for path, count in paths:
            # Tạo tập hợp các item trong path
            path_items = {node[0] for node in path}

            # Kiểm tra xem tất cả các item trong prefix có nằm trong path không
            if all(item in path_items for item in prefix):
                sum_twu= 0
                # Tính expected support cho path bằng cách nhân các xác suất (prob) của các items trong prefix
                for item in prefix:
                    # Tìm item trong path
                    i = next((i for i in path if i[0] == item), None)
                    if i:
                        sum_twu = i[3]  # Nhân xác suất (prob) của item trong path

                prefix_twu += sum_twu * count  # Nhân với count của path

        return prefix_twu


    def mine_top_k_patterns(self, uhufp_tree, k, minSup, minUtil, expected_support, prefix=None, top_k=None):
        if top_k is None:
            top_k = []
        if prefix is None:
            prefix = []
        if len(top_k) == k and top_k is not None:
            minSup = top_k[-1][1]

        # Iterate over items in the header table in reverse order
        for item, nodes in uhufp_tree.header_table.items():
            if item not in prefix:
                max_prob = max(node.prob for node in nodes if node.item == item)
                overestimate = max_prob*expected_support
                if overestimate < minSup:
                    return top_k
                # Create a new prefix by appending the current item
                new_prefix = [item] + prefix

                if any(existing_pattern[0] == tuple(sorted(new_prefix)) for existing_pattern in top_k):
                    continue
                print(f"ITEM: {new_prefix}")
                # Build conditional pattern base and tree
                conditional_pattern_base, miu, path_util_item = self.build_conditional_pattern_base(uhufp_tree, new_prefix, minUtil)
                if path_util_item >= minUtil:
                    # Calculate the expected support using the conditional pattern tree
                    expected_support = self.calculate_expected_support(conditional_pattern_base, new_prefix)
            
                    # Calculate the utility for the new prefix using the conditional pattern tree
                    utility = self.calculate_utility(conditional_pattern_base, new_prefix)

                    conditional_pattern_tree = self.build_conditional_tree(conditional_pattern_base, miu)
                    conditional_pattern_tree.print_tree()
                    conditional_pattern_tree.print_header_table()
                    
                
                    
                    # Print debugging information
                    print(f"{new_prefix} : {expected_support} : {utility}")
                    
                    # Check if the expected support meets the minimum support threshold
                    if expected_support > minSup:
                        # Check if the utility meets the minimum utility threshold
                        if utility >= minUtil:
                            # Add the pattern to top_k list if necessary
                            top_k.append((tuple(sorted(new_prefix)), expected_support, utility))
                            # Sort and maintain top-k
                            top_k.sort(key=lambda x: (x[1], x[2]), reverse=True)
                            if len(top_k) > k:
                                top_k.pop()
                            if len(top_k) == k:
                                minSup = top_k[-1][1]
                            # print("\nTop-K High Utility Frequent Patterns:")
                            # for i, (pattern, expSup, utility) in enumerate(top_k, 1):
                                # print(f"{i}. Pattern: {pattern}, ExpSup: {expSup}, Utility: {utility}")
                            # print("*"*50)
                        # Recursively explore further patterns by calling the function on the conditional pattern tree
                        # if self.calculate_twu(conditional_pattern_base, new_prefix) >= minUtil:
                        if conditional_pattern_tree.root.children:
                            self.mine_top_k_patterns(conditional_pattern_tree, k, minSup, minUtil, expected_support, new_prefix, top_k)
    
        return top_k



    def collect_paths(self, uhufp_tree, prefix: List[str]) -> List[Tuple[List[Tuple[str, float, float, float]], int]]:
        paths = []
        
        # Lấy item cuối cùng của prefix
        last_item = prefix[-1] if prefix else None
        # print(f"LAST ITEM: {last_item}")
        if last_item and last_item in uhufp_tree.header_table:
            for node in uhufp_tree.header_table[last_item]:
                path = []
                current_node = node
                
                # Theo dõi từ node hiện tại về gốc của cây
                while current_node and current_node.item is not None:
                    path.append((current_node.item, current_node.prob, current_node.utility, current_node.nu))
                    if current_node.parent is None:
                        break
                    current_node = current_node.parent
                
                # Đảo ngược path để có thứ tự từ gốc đến node hiện tại
                path.reverse()
                
                # Kiểm tra xem path có chứa tất cả các item trong prefix không
                if all(item in [p[0] for p in path] for item in prefix):
                    paths.append((path, node.count))
        # for path, count in paths:
            # print(f"Path: {path}, Count: {count}")
        return paths



    def build_conditional_pattern_base(self, uhufp_tree, item: str, min_util: float) -> List[Tuple[Dict[str, float], Dict[str, float], int]]:
        paths = self.collect_paths(uhufp_tree, item)
        conditional_pattern_base = []

        # print(f"Paths for item {item}: {paths}")
        path_utilities_item = defaultdict(int)
        for path, count in paths:
            path_utility = path[-1][3]  # Total utility of the path

            # Calculate the total TWU of items in the path
            for i in path:
                path_utilities_item[i[0]] += path_utility

            # print(f"Path: {path}")
            # print(f"Path utility: {path_utility}")
            # print(f"Path utilities item: {dict(path_utilities_item)}")
        path_util_item = 0
        for i in item:
            path_util_item += path_utilities_item[i]
        unpromissing_item = []
        miu = {}
        for path, count in paths:
            for i, s, u, _ in path:
                if i in miu:
                    # So sánh giá trị u hiện tại với giá trị nhỏ nhất đã lưu và cập nhật nếu cần
                    miu[i] = min(miu[i], u)
                else:
                    # Nếu i chưa tồn tại trong miu, khởi tạo nó với giá trị u hiện tại
                    miu[i] = u

        for path, count in paths:
            filtered_pu = 0
            for i, s, _, u in path:
                # Apply DLU strategy: subtract min item utility * path count from path utility
                if path_utilities_item[i] <= min_util:
                    filtered_pu += miu[i] * count  # Adjust path utility
                    print(f"Item: {i}, Node Utility: {u}, Min Utility: {miu[i]}, Count: {count}")
                    print(f"Filtered Utility for item {i}: {filtered_pu}")
                    unpromissing_item.append((i,s,u))  
            
            reorganized_path = []

            for i, s, u, _ in path:
                if (i,s,u) not in unpromissing_item:
                    reorganized_path.append((i, s, u, path[-1][3] - filtered_pu))
            # Sort reorganized path based on filtered utility
            sorted_reorganized_path = sorted(reorganized_path, key=lambda x: path_utilities_item[x[0]], reverse=False)
            print(f"Reorganized Path: {reorganized_path}")
            # print(f"Sorted Reorganized Path: {sorted_reorganized_path}")
        
            if sorted_reorganized_path:
                conditional_pattern_base.append((sorted_reorganized_path, count))
                # print(f"Conditional Pattern Base Entry: {conditional_pattern_base}")
        
        return conditional_pattern_base, miu, path_util_item



    def build_conditional_tree(self, conditional_pattern_base: List[Tuple[Tuple[str,float,int], int]], miu) -> 'UHUFP_tree':
        conditional_tree = UHUFP_tree()

        for path, count in conditional_pattern_base:
            filtered_path = {item: support for item, support,_, _ in path}
            filtered_utilities = {item: util for item, _, util, _ in path}
            filtered_path_utility = {item: pu for item, _, _, pu in path}
            if filtered_path:  # Ensure non-empty paths
                print(f"Inserting transaction into conditional tree: Path: {filtered_path}, Utilities: {filtered_utilities}")
                conditional_tree.insert_reorganized_path(filtered_path, filtered_utilities, filtered_path_utility, miu)

        return conditional_tree


    def load_transactions_and_utilities(self, transaction_file: str, utility_file: str) -> Tuple[List[Dict[str, float]], List[Dict[str, float]], List[str]]:
        transactions = []
        utilities = []
        items = []
        dbUtil = 0
        item_twu = defaultdict(int)
        with open(transaction_file, mode='r') as tfile, open(utility_file, mode='r') as ufile:
            tlines = tfile.readlines()
            ulines = ufile.readlines()

            # Process transaction file
            header = tlines[0].strip().split()
            items = [item.strip() for item in header]

            for line1, line2 in zip(tlines[1:], ulines):
                row = line1.strip().split()
                transaction = {items[i]: float(row[i]) for i in range(len(items)) if float(row[i]) > 0}
                transactions.append(transaction)

                parts = line2.strip().split(':')
                item_list = parts[0].split()
                utility_values = list(map(int, parts[2].split()))
                utility = {item_list[i]: utility_values[i] for i in range(len(item_list)) if item_list[i] in transaction}
                utilities.append(utility)
                dbUtil += int(parts[1])
                # print(transaction_utility)
                self.database_size += 1

            for utility in utilities:
                for item, util in utility.items():
                    item_twu[item] += sum(utility.values())
        
        return transactions, utilities, item, item_twu, dbUtil

    def build_tree_and_mine_patterns(self, transaction_file: str, utility_file: str, k: int, percentage: float, minSup: float):
        
        self.start_timestamp = time.time()
        self.candidates = 0
        self.database_size = 0
        transactions, utilities, items, item_twu, dbUtil = self.load_transactions_and_utilities(transaction_file, utility_file)
        


        promising_items = set()
        unpromising_items = set()
        reorganized_transaction = defaultdict(float)
        reorganized_utilities = defaultdict(int)

        
        uhufp_tree = UHUFP_tree()
        minUtil = int(dbUtil*percentage)

        for item, twu in item_twu.items():
            if twu >= self.minUtil:
                promising_items.add(item)
            else:
                unpromising_items.add(item)

        # print(promising_items)

        for transaction, utility in zip(transactions, utilities):
            # print(transaction)
            #stratagy 1
            reorganized_transaction = {item: prob for item, prob in transaction.items() if item in promising_items}
            reorganized_utilities = {item: util for item, util in utility.items() if item in reorganized_transaction}
            sorted_items = sorted(reorganized_transaction.keys(), key=lambda item: item_twu[item], reverse=True)
            # print(sorted_items)
            uhufp_tree.insert_reorganized_transaction(reorganized_transaction, reorganized_utilities, sorted_items)

        # miut = {}
        
        # for item, nodes in uhufp_tree.header_table.items():
        #     miut[item] = min(node.utility for node in nodes)
        # uhufp_tree.print_header_table()
        uhufp_tree.prune_patterns(k, item_twu, minUtil)
        uhufp_tree.prune_tree(uhufp_tree.root)
        

        # print(miut)
        # print({'database util: ': dbUtil})
        print({'min util: ': minUtil})
        # Print updated header table
        # print("\nFiltered Header Table:")
        # uhufp_tree.print_header_table()
        # print(len(uhufp_tree.header_table))

        # Bắt đầu theo dõi bộ nhớ
        tracemalloc.start()
        self.top_k_UHUFP = self.mine_top_k_patterns(uhufp_tree, k, minSup, minUtil, 1.0)
        self.minUtil = minUtil
        # Kết thúc theo dõi bộ nhớ và lấy thông tin
        _, self.peak_memory_usage = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        self.peak_memory_usage /= 10**6  # Chuyển đổi sang MB

        self.end_timestamp = time.time()
        # print("\nTop-K High Utility Frequent Patterns:")
        for i, (pattern, expSup, utility) in enumerate(self.top_k_UHUFP, 1):
            print(f"{i}. Pattern: {pattern}, ExpSup: {expSup}, Utility: {utility}")

    def print_stats(self, path):
        with open(path, 'w') as output_file:
            output_file.write(f"minUtil: {self.minUtil}\n")
            for i, (pattern, expSup, utility) in enumerate(self.top_k_UHUFP, 1):
                output_file.write(f"{i}. Pattern: {pattern}, ExpSup: {expSup}, Utility: {utility}\n")
            output_file.write("=============  TOP-K UFPs v1.20 - STATS =============\n")
            output_file.write(f" Transactions count from database : {self.database_size}\n")
            output_file.write(f" Candidates count : {self.candidates}\n")
            output_file.write(f" Algorithm run time : {self.end_timestamp - self.start_timestamp:.0f} seconds\n")
            output_file.write(f" Peak memory usage : {self.peak_memory_usage:.2f} MB\n")  # Ghi lại peak memory usage
# Example usage

In [6]:
transaction_file = '../../data/example.txt'
utility_file = '../../data/example_utility.txt'
k = 6
percentage = 0.3
minSup = 0.0
tuhufp_growth = TUHUFP_Growth()
tuhufp_growth.build_tree_and_mine_patterns(transaction_file, utility_file, k, percentage, minSup)
tuhufp_growth.print_stats("../../out/TUHUFP_Growth/output_example.txt")

{'min util: ': 30}
ITEM: ['c']
Item: c, Node Utility: 1, Min Utility: 1, Count: 1
Filtered Utility for item c: 1
Reorganized Path: []
Item: c, Node Utility: 3, Min Utility: 1, Count: 1
Filtered Utility for item c: 1
Reorganized Path: []
Item: c, Node Utility: 6, Min Utility: 1, Count: 1
Filtered Utility for item c: 1
Reorganized Path: []
Item: c, Node Utility: 5, Min Utility: 1, Count: 1
Filtered Utility for item c: 1
Reorganized Path: []
Item: c, Node Utility: 3, Min Utility: 1, Count: 1
Filtered Utility for item c: 1
Reorganized Path: []
ITEM: ['d']
Reorganized Path: [('c', 0.9, 1, 6), ('d', 0.6, 5, 6)]
Reorganized Path: [('c', 0.7, 3, 8), ('d', 0.6, 5, 8)]
Reorganized Path: [('c', 0.8, 6, 11), ('d', 0.9, 5, 11)]
Reorganized Path: [('c', 0.9, 3, 13), ('d', 0.3, 10, 13)]
Reorganized Path: [('d', 0.9, 5, 5)]
Inserting transaction into conditional tree: Path: {'c': 0.9, 'd': 0.6}, Utilities: {'c': 1, 'd': 5}
Inserting transaction into conditional tree: Path: {'c': 0.7, 'd': 0.6}, Utilit

In [3]:
transaction_file = '../../data/input_foodmart.txt'
utility_file = '../../data/foodmart_utility.txt'
k = 900
percentage = 0.0004
minSup = 0.0
tuhufp_growth = TUHUFP_Growth()
tuhufp_growth.build_tree_and_mine_patterns(transaction_file, utility_file, k, percentage, minSup)
tuhufp_growth.print_stats("../../out/TUHUFP_Growth/output_foodmart_top_900.txt")

{'min util: ': 4804}
ITEM: ['304']
Reorganized Path: [('304', 0.94, 378, 378)]
Reorganized Path: [('304', 0.73, 378, 378)]
Reorganized Path: [('304', 0.12, 567, 567)]
Reorganized Path: [('304', 0.78, 567, 567)]
Reorganized Path: [('304', 0.04, 567, 567)]
Reorganized Path: [('304', 0.96, 567, 567)]
Reorganized Path: [('304', 0.11, 756, 756)]
Reorganized Path: [('304', 0.5, 378, 378)]
Reorganized Path: [('304', 0.42, 756, 756)]
Reorganized Path: [('304', 0.56, 567, 567)]
Reorganized Path: [('304', 0.86, 756, 756)]
Reorganized Path: [('304', 0.65, 567, 567)]
Reorganized Path: [('304', 0.49, 567, 567)]
Reorganized Path: [('304', 0.76, 378, 378)]
Reorganized Path: [('304', 0.98, 189, 189)]
Reorganized Path: [('304', 0.99, 567, 567)]
Reorganized Path: [('304', 0.79, 567, 567)]
Reorganized Path: [('304', 0.68, 567, 567)]
Reorganized Path: [('304', 0.71, 944, 944)]
Reorganized Path: [('304', 0.45, 756, 756)]
Reorganized Path: [('304', 0.95, 756, 756)]
Reorganized Path: [('304', 0.61, 756, 756)

KeyboardInterrupt: 

In [ ]:
transaction_file = '../../data/input_retail.txt'
utility_file = '../../data/retail_utility.txt'
k = 100
percentage = 0.001
minSup = 0.0
tuhufp_growth = TUHUFP_Growth()
tuhufp_growth.build_tree_and_mine_patterns(transaction_file, utility_file, k, percentage, minSup)
tuhufp_growth.print_stats("../../out/TUHUFP_Growth/output_retail_top_100.txt")

KeyboardInterrupt: 

In [ ]:
# transaction_file = '../../data/input_chess.txt'
# utility_file = '../../data/chess_utility.txt'
# k = 300
# percentage = 0.002
# minSup = 0.0
# tuhufp_growth = TUHUFP_Growth()
# tuhufp_growth.build_tree_and_mine_patterns(transaction_file, utility_file, k, percentage, minSup)
# tuhufp_growth.print_stats("../../out/TUHUFP_Growth/output_chess_top_300.txt")